In [1]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp

# =====================================
params = get_settings(scale_to_mm=1000)
loader = Loader()
# =====================================

# def execute(frame):
#     global params, loader
#     _start = time()
#     Im, Y, Calib = epfl_campus.get(root, frame)
#     heatmaps, pafs = loader.load_confidence_map_and_paf(
#         'campus', Im, frame, dir=Settings['tmp'])
#     _end = time()
#     print('frame ' + str(frame) + ', elapsed:', _end - _start)

    
def generate_pcp_score(frame):
    global Settings, params
    alpha = 0.5
    
    Im, Y, Calib = epfl_campus.get(root, frame)
    heatmaps, pafs = loader.load_confidence_map_and_paf(
        'campus', Im, frame, dir=Settings['tmp'])
    detections = pose.estimate(Calib, heatmaps, pafs, 
                               settings=params, debug=False)
    
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
  
    for gtid, gt in enumerate(Y):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    del heatmaps
    del pafs
    del detections
    del Im
    del Y
    del Calib
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs

# =====================================

valid_frames = list(range(350, 470)) + list(range(650, 750))
PER_GTID = {}
for frame in valid_frames:
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs =\
        generate_pcp_score(frame)
    _end = time()
    print('frame ' + str(frame) + ', elapsed:', _end - _start)
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


frame 350, elapsed: 1.7805025577545166
frame 351, elapsed: 1.0495445728302002
frame 352, elapsed: 1.05680513381958
frame 353, elapsed: 1.0631353855133057
frame 354, elapsed: 1.0676279067993164
frame 355, elapsed: 1.085660696029663
frame 356, elapsed: 1.135779619216919
frame 357, elapsed: 1.0694451332092285
frame 358, elapsed: 1.0535452365875244
frame 359, elapsed: 1.058398723602295
frame 360, elapsed: 1.0668272972106934
frame 361, elapsed: 1.0871694087982178
frame 362, elapsed: 1.1099514961242676
frame 363, elapsed: 1.0671470165252686
frame 364, elapsed: 1.073176622390747
frame 365, elapsed: 1.098907709121704
frame 366, elapsed: 1.1087446212768555
frame 367, elapsed: 1.0792734622955322
frame 368, elapsed: 1.0952551364898682
frame 369, elapsed: 1.0710194110870361
frame 370, elapsed: 1.0814199447631836
frame 371, elapsed: 1.0550909042358398
frame 372, elapsed: 1.0567045211791992
frame 373, elapsed: 1.0287950038909912
frame 374, elapsed: 1.0478603839874268
frame 375, elapsed: 1.0318362712

frame 742, elapsed: 1.4882049560546875
frame 743, elapsed: 1.3570005893707275
frame 744, elapsed: 1.3393175601959229
frame 745, elapsed: 1.3204193115234375
frame 746, elapsed: 2.2546114921569824
frame 747, elapsed: 1.402238368988037
frame 748, elapsed: 2.2399985790252686
frame 749, elapsed: 1.296623706817627


In [2]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

actor  1
	uarms: 0.9787234042553191
	larms: 0.601063829787234
	ulegs: 1.0
	llegs: 0.9973404255319149
	avg:   0.894281914893617
actor  2
	uarms: 0.8897058823529411
	larms: 0.6544117647058824
	ulegs: 0.9669117647058824
	llegs: 0.9669117647058824
	avg:   0.869485294117647
actor  0
	uarms: 0.6770833333333334
	larms: 0.5104166666666666
	ulegs: 0.9375
	llegs: 0.9375
	avg:   0.765625

avg*:   0.843130736337088


## Same as Master thesis
* actor  1
    * uarms: 0.9813829787234043
    * larms: 0.601063829787234
	* ulegs: 1.0
	* llegs: 0.9973404255319149
	* avg:   0.8949468085106383
* actor  2
	* uarms: 0.9007352941176471
	* larms: 0.6544117647058824
	* ulegs: 0.9742647058823529
	* llegs: 0.9742647058823529
	* avg:   0.8759191176470588
* actor  0
	* uarms: 0.6979166666666666
	* larms: 0.5104166666666666
	* ulegs: 0.9583333333333334
	* llegs: 0.9583333333333334
	* avg:   0.78125
* avg*:   0.8507053087192323

## Same as Master thesis without Unaries
* actor  1
	* uarms: 0.9787234042553191
	* larms: 0.601063829787234
	* ulegs: 1.0
	* llegs: 0.9973404255319149
	* avg:   0.894281914893617
* actor  2
	* uarms: 0.8897058823529411
	* larms: 0.6544117647058824
	* ulegs: 0.9669117647058824
	* llegs: 0.9669117647058824
	* avg:   0.869485294117647
* actor  0
	* uarms: 0.6770833333333334
	* larms: 0.5104166666666666
	* ulegs: 0.9375
	* llegs: 0.9375
	* avg:   0.765625
* avg*:   0.843130736337088